In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from fbprophet import Prophet

#Getting Data
import quandl as qdl
import requests
import json
from pandas.io.json import json_normalize 
import seaborn as sns

In [ ]:
ETH_2h_url = 'https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=USD&limit=2000&aggregate=2&e=CCCAGG'
ETH_2h_response = requests.get(ETH_2h_url)
if (ETH_2h_response.status_code == 200):
    ETH_2h = (ETH_2h_response.json())
    
ETH_2h = json_normalize(ETH_2h['Data'])
ETH_2h['time'] = pd.to_datetime(ETH_2h['time'], unit='s')
ETH_2h_noIdenx = ETH_2h.set_index('time')


In [ ]:
ETH_2h.tail()

Get start and end date of returned Crypto Data

In [ ]:
last = len(ETH_2h) - 1
firstDate = ETH_2h['time'].iloc[0]
lastDate =  ETH_2h['time'].iloc[last]
lastDate

In [ ]:
prohpetStructure = ETH_2h[['close', 'time']]
prohpetStructure = prohpetStructure.rename(columns={'close': 'y', 'time': 'ds'})
prohpetStructure['y'] = np.log(prohpetStructure['y'])

In [ ]:
m = Prophet(changepoint_prior_scale=0.005, yearly_seasonality=True)
m.fit(prohpetStructure)
future = m.make_future_dataframe(periods=10, freq='2h')
forecast = m.predict(future)
#This is causing issues for soe reason. Or maybe its right. It is plotting the lines at different levels
forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
m.plot(forecast)
for changepoint in m.changepoints:
    plt.axvline(changepoint,ls='--', lw=1)

In [ ]:
deltas = m.params['delta'].mean(0)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111)
ax.bar(range(len(deltas)), deltas)
ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
ax.set_ylabel('Rate change')
ax.set_xlabel('Potential changepoint')
fig

In [ ]:
import tweepy as twpy
from textblob import TextBlob

In [ ]:
consumer_key = 'fba8sGRIIg0EoQOtfgxHGv7EI'
consumer_secret = 'z8LKzW9epwOPWsYfE1QXIApFlo1VrbM6rGP9K7zdr9tXa3TocS'
access_token = '893107128092606465-r9EFEkdoAoQyuHEOSMrQMdvUG7rAOBO'
access_token_sedret = 'ZeGap5yg7GF7WdJ31wG2vb3NxVALCH5PUJj6hsCPPOv5e'
auth = twpy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_sedret)
api = twpy.API(auth)

I searched for most reputable Crypto news sources and have grabbed there tweets going back as far as possible. I plan to aggregate them by date time test for sentiment and average result across all source on a two hour basis. 

In [ ]:
users = ['BTCTN', 'kyletorpey', 'hodlhodl', 'bravenewcoin', 'coindesk', 'Cointelegraph']
dates = []
tweets = []
idString = []
userName = []
for i in range(len(users)):
    user = users[i]
    for status in twpy.Cursor(api.user_timeline, screen_name=user ).items(2900):
        dates.append(status.created_at)
        tweets.append(status.text)
        userName.append(status.user.name)

In [ ]:
combinedTweets = pd.DataFrame({
    'dates': dates,
    'tweets': tweets,
    'user': userName
})    
combinedTweets = combinedTweets.sort_values('dates')
combinedTweets.tail()

Perform Sentiment Analysis using TextBlob

In [ ]:
combinedTweets['sentiment-polarity'] = combinedTweets['tweets'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
combinedTweets['sentiment-subjectivity'] = combinedTweets['tweets'].apply(lambda tweet: TextBlob(tweet).sentiment.subjectivity)


In [ ]:
tweetTimeSentiment = combinedTweets[['dates', 'sentiment-polarity', 'sentiment-subjectivity']]
tweetTimeSentiment['dates'] = pd.to_datetime(tweetTimeSentiment['dates'])
tweetTimeSentiment = tweetTimeSentiment.resample(rule='120Min', on='dates').mean()
tweetTimeSentiment = tweetTimeSentiment.fillna(0)
mask = (tweetTimeSentiment.index > firstDate) & (tweetTimeSentiment.index <= lastDate)
tweetTimeSentiment = tweetTimeSentiment.iloc[mask]
tweetTimeSentiment.head()

In [ ]:
tweetTimeSentiment.head()

In [ ]:
last = len(ETH_2h) - 1
firstDate = ETH_2h['time'].iloc[0]
lastDate =  ETH_2h['time'].iloc[last]



tweetTimeSentiment = combinedTweets[['dates', 'sentiment-polarity', 'sentiment-subjectivity']]
tweetTimeSentiment['dates'] = pd.to_datetime(tweetTimeSentiment['dates'])
tweetTimeSentiment = tweetTimeSentiment.resample(rule='120Min', on='dates').mean()
tweetTimeSentiment = tweetTimeSentiment.fillna(0)
mask = (tweetTimeSentiment.index > firstDate) & (tweetTimeSentiment.index <= lastDate)
tweetTimeSentiment = tweetTimeSentiment.iloc[mask]
tweetTimeSentiment.head()





BTCHistPriceSentiment = pd.merge(ETH_2h_noIdenx, tweetTimeSentiment, how='inner', left_index=True, right_index=True)
BTCHistPriceSentiment['sma5'] = BTCHistPriceSentiment['close'].rolling(5).mean()
BTCHistPriceSentiment['sma10'] = BTCHistPriceSentiment['close'].rolling(10).mean()
BTCHistPriceSentiment['sma15'] = BTCHistPriceSentiment['close'].rolling(15).mean()
BTCHistPriceSentiment['sma25'] = BTCHistPriceSentiment['close'].rolling(25).mean()
BTCHistPriceSentiment['sma50'] = BTCHistPriceSentiment['close'].rolling(50).mean()
BTCHistPriceSentiment = BTCHistPriceSentiment.reset_index()

BTCHistPriceSentiment.tail(5)

In [ ]:
prohpetStructure = ETH_2h[['close', 'time']]
prohpetStructure = prohpetStructure.rename(columns={'close': 'y', 'time': 'ds'})
prohpetStructure['y'] = np.log(prohpetStructure['y'])

In [ ]:
m = Prophet(changepoint_prior_scale=0.01, yearly_seasonality=True)
m.fit(prohpetStructure)
future = m.make_future_dataframe(periods=20, freq='2h')
forecast = m.predict(future)
#This is causing issues for soe reason. Or maybe its right. It is plotting the lines at different levels
forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
m.plot(forecast)
for changepoint in m.changepoints:
    plt.axvline(changepoint,ls='--', lw=1)

In [ ]:
forecast.tail()

Combine Forecast with Sentiment, Price, and SMA DF

In [ ]:

lastDateWPrediction =  forecast['ds'].iloc[last]
mask = (tweetTimeSentiment.index > firstDate) & (tweetTimeSentiment.index <= lastDateWPrediction)


forecast.iloc[mask]
future = forecast.tail(20)

Sentiment, Price, SMA, With Forecast


In [ ]:
BTCSentimentForecast = pd.merge(forecast, BTCHistPriceSentiment, how='inner', left_index=True, right_index=True)
BTCSentimentForecast = BTCSentimentForecast.append(future)
BTCSentimentForecast.tail(5)

In [ ]:

sentiment = tweetTimeSentiment[['sentiment-polarity', 'sentiment-subjectivity']]
sentiment = sentiment.tail(20)

sentiment.plot(figsize=[10,7])


In [ ]:
BTCSentimentForecastTimeIndex = BTCSentimentForecast.set_index('ds')
BTCSentimentForecastTimeIndex.tail()

In [ ]:
BTCSentimentForecastTimeIndex = BTCSentimentForecastTimeIndex[['yhat']]
BTCSentimentForecastTimeIndex = BTCSentimentForecastTimeIndex.tail(100)
BTCSentimentForecastTimeIndex.plot(figsize=[10,7])